In [10]:
file_path = './train'

In [11]:
import re
import os
file_number = []
for filename in os.listdir(file_path):
    if filename.endswith(".txt") :
        file_number.append(re.findall(r'\d+', filename)[0])
        
file_number = list(set(file_number))
print(len(file_number))
print(file_number[0:5])

293
['765982381', '727736557', '741655444', '762546428', '696735702']


In [12]:
article_sentence,sentence_number,gold_labels = ([],[],[])
article_dic = {}
sent_cnt = 0
propaganda_cnt = 0

propaganda_list = []
non_propaganda_list = []

for file_num in file_number:
    article_dic[file_num] = {}
    with open(file_path+"/"+"article"+file_num+".txt", "r", encoding='utf-8') as f:
        count=1
        for line in f.readlines():
            #line = line.strip()
            #if len(line)>0:
            article_dic[file_num][str(count)] = {'text':line.strip()}
            count+=1
    #print("length of article %s is %d" % (file_num, len(article_dic[file_num])))
    sent_cnt += len(article_dic[file_num])
    with open(file_path+"/"+"article"+file_num+".task2.labels", "r", encoding='utf-8') as f:
        for line in f.readlines():
            article_id, sentence_id, gold_label = line.rstrip().split("\t")
            if gold_label=='propaganda':
                propaganda_cnt+=1
            if article_dic[str(article_id)][str(sentence_id)]['text']=='':
                continue
            if str(sentence_id) not in article_dic[str(article_id)]:
                print("not found: %s" % (str(sentence_id)))
            else:
                article_dic[str(article_id)][str(sentence_id)]['label'] = gold_label
                if gold_label=='propaganda':
                    propaganda_list.append((article_dic[str(article_id)][str(sentence_id)]['text'],'propaganda'))
                else:
                    non_propaganda_list.append((article_dic[str(article_id)][str(sentence_id)]['text'],'non_propaganda'))
            
print("There are %d articles in total." % len(article_dic))
print("There are %d sentences in total." % sent_cnt)
print("There are %d propaganda sentences in total." % len(propaganda_list))
print("There are %d non-propaganda sentences in total." % len(non_propaganda_list))
#print("Among them, there are %d sentences labeled as propaganda.\n" % gold_labels.count('propaganda'))
#print("Example sentence:\n", sent_dic[2])
#print("Label:", gold_labels[2])

There are 293 articles in total.
There are 15170 sentences in total.
There are 3938 propaganda sentences in total.
There are 10325 non-propaganda sentences in total.


In [13]:
sentences = propaganda_list+non_propaganda_list

from sklearn.model_selection import train_test_split
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)
test_sentences, dev_sentences = train_test_split(test_sentences, test_size=0.33, random_state=42)

print("length of train set: %d"%(len(train_sentences)))
print("length of dev set: %d"%(len(dev_sentences)))
print("length of test set: %d"%(len(test_sentences)))

length of train set: 11410
length of dev set: 942
length of test set: 1911


In [14]:
cnt=0
for item in train_sentences:
    if item[1]=='propaganda':
        cnt+=1
print("train set propaganda label: %d"%(cnt))
cnt=0
for item in dev_sentences:
    if item[1]=='propaganda':
        cnt+=1
print("dev set propaganda label: %d"%(cnt))
cnt=0
for item in test_sentences:
    if item[1]=='propaganda':
        cnt+=1
print("test set propaganda label: %d"%(cnt))

train set propaganda label: 3143
dev set propaganda label: 263
test set propaganda label: 532


In [15]:
x_train = []
y_train = []
x_test = []
y_test = []

#from nltk.tokenize import word_tokenize

for item in train_sentences:
    x_train.append(item[0])
    y_train.append(item[1])
        
for item in test_sentences:
    x_test.append(item[0])
    y_test.append(item[1])

print(len(x_train))
print(len(x_test))

11410
1911


In [16]:
import nltk
import sys
import xgboost as xgb
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [8]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, max_features=None, 
                             strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1)
train = vectorizer.fit_transform(x_train)
print("n_samples: %d, n_features: %d" % train.shape)
test = vectorizer.transform(x_test)
#print(test)
#feature_name = vectorizer.get_feature_names()
print("Checking that the number of features in train and test correspond: %s - %s" % (train.shape[1],test.shape[1]))
#print(type(train))

n_samples: 11410, n_features: 28496
Checking that the number of features in train and test correspond: 28496 - 28496


In [22]:
#SVM
model = LinearSVC(C=1.0, class_weight='balanced', multi_class='ovr', random_state=40)
model.fit(train, y_train)
predictions = model.predict(test)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.81909161 0.48529412]
recall: [0.7715736  0.55827068]
fscore: [0.79462285 0.51923077]
support: [1379  532]
accuracy: 0.7121925693354265


In [23]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression 
 
classifier = LogisticRegression() 
classifier.fit(train, y_train)  

predictions = classifier.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.75988539 0.68072289]
recall: [0.96156635 0.21240602]
fscore: [0.84891165 0.32378223]
support: [1379  532]
accuracy: 0.7530088958660387


In [24]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(oob_score=True, n_estimators = 8)
rf.fit(train, y_train)
predictions = rf.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74577227 0.59124088]
recall: [0.95939086 0.15225564]
fscore: [0.83920076 0.24215247]
support: [1379  532]
accuracy: 0.7346938775510204


/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [25]:
rf_20 = RandomForestClassifier(oob_score=True, n_estimators = 20)
rf_20.fit(train, y_train)
predictions = rf_20.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74431503 0.65740741]
recall: [0.97316896 0.13345865]
fscore: [0.84349466 0.221875  ]
support: [1379  532]
accuracy: 0.7394034536891679


/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [26]:
rf_100 = RandomForestClassifier(oob_score=True, n_estimators = 100)
rf_100.fit(train, y_train)
predictions = rf_100.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74055829 0.69047619]
recall: [0.98114576 0.10902256]
fscore: [0.84404242 0.18831169]
support: [1379  532]
accuracy: 0.7383568812140241


In [14]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
best_k=-1
best_score=0
best_method=''
for method in ['uniform','distance']:
    for i in range(1,11):
        knn_clf=KNeighborsClassifier(n_neighbors=i,weights=method)
        knn_clf.fit(train,y_train)
        scores=knn_clf.score(test,y_test)
        if scores>best_score:
            best_score=scores
            best_k=i
            best_method=method

print('best k:%d, best score:%.4f, best method: %s'%(best_k,best_score,best_method))

best k:4, best score:0.7310, best method: distance


In [27]:
knn_clf=KNeighborsClassifier(n_neighbors=4,weights='distance')
knn_clf.fit(train,y_train)
predictions = knn_clf.predict(test) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.73265196 0.67307692]
recall: [0.98767223 0.06578947]
fscore: [0.84126004 0.11986301]
support: [1379  532]
accuracy: 0.7310308738880168


In [21]:
cnt=0
for i in range(len(predictions)):
    if predictions[i]=='propaganda':
        cnt+=1
print(cnt)

52


In [30]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()

vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, max_features=None, 
                             strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1)
x_train_vec = vectorizer.fit_transform(x_train).toarray()
print(x_train_vec.shape)

ner_list = ['PERSON','LOC','ORG','DATE','TIME','NORP','PRODUCT','EVENT','PERCENT','QUANTITY','ORDINAL','CARDINAL']
features = []
for i, item in enumerate(x_train):
    feature = []
    doc = nlp(item)
    doc = [X.label_ for X in doc.ents]
    for ner in ner_list:
        if ner in doc:
            feature.append(1)
        else:
            feature.append(0)
    features.append(feature)
features = np.array(features)
x_train_vec = np.concatenate((x_train_vec, features), axis=1)
print(x_train_vec.shape)

(11410, 28370)
(11410, 28382)


In [31]:
x_test_vec = vectorizer.transform(x_test).toarray()
print(x_test_vec.shape)

ner_list = ['PERSON','LOC','ORG','DATE','TIME','NORP','PRODUCT','EVENT','PERCENT','QUANTITY','ORDINAL','CARDINAL']
features = []
for i, item in enumerate(x_test):
    feature = []
    doc = nlp(item)
    doc = [X.label_ for X in doc.ents]
    for ner in ner_list:
        if ner in doc:
            feature.append(1)
        else:
            feature.append(0)
    features.append(feature)
features = np.array(features)
x_test_vec = np.concatenate((x_test_vec, features), axis=1)
print(x_test_vec.shape)

(1911, 28370)
(1911, 28382)


In [32]:
#SVM using NER feature
model = LinearSVC(C=1.0, class_weight='balanced', multi_class='ovr', random_state=40)
model.fit(x_train_vec, y_train)
predictions = model.predict(x_test_vec)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.84034268 0.5215311 ]
recall: [0.78245105 0.61466165]
fscore: [0.81036425 0.56427955]
support: [1379  532]
accuracy: 0.7357404500261643


In [33]:
#LogisticRegression using NER feature
from sklearn.linear_model import LogisticRegression 
 
classifier = LogisticRegression() 
classifier.fit(x_train_vec, y_train)  

predictions = classifier.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.77381648 0.725     ]
recall: [0.96011603 0.27255639]
fscore: [0.85695793 0.39617486]
support: [1379  532]
accuracy: 0.7687074829931972


/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [35]:
#RandomForest_100 using NER feature
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf_100 = RandomForestClassifier(oob_score=True, n_estimators = 100)
rf_100.fit(x_train_vec, y_train)
predictions = rf_100.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74890351 0.85057471]
recall: [0.99057288 0.13909774]
fscore: [0.85295036 0.23909532]
support: [1379  532]
accuracy: 0.7535321821036107


In [36]:
#KNN using NER feature
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors=4,weights='distance')
knn_clf.fit(x_train_vec,y_train)
predictions = knn_clf.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.75653207 0.53744493]
recall: [0.92385787 0.22932331]
fscore: [0.83186419 0.32147563]
support: [1379  532]
accuracy: 0.7305075876504448


In [40]:
from nltk.sentiment import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

x_train_vec = vectorizer.fit_transform(x_train).toarray()
print(x_train_vec.shape)
x_test_vec = vectorizer.transform(x_test).toarray()
print(x_test_vec.shape)

features = []
for i, item in enumerate(x_train):
    polarity_score = vader_analyzer.polarity_scores(item)
    feature = [score for (name, score) in polarity_score.items()]
    features.append(feature)
    
features = np.array(features)
x_train_vec = np.concatenate((x_train_vec, features), axis=1)
print(x_train_vec.shape)

features = []
for i, item in enumerate(x_test):
    polarity_score = vader_analyzer.polarity_scores(item)
    feature = [score for (name, score) in polarity_score.items()]
    features.append(feature)
    
features = np.array(features)
x_test_vec = np.concatenate((x_test_vec, features), axis=1)
print(x_test_vec.shape)

(11410, 28370)
(1911, 28370)
(11410, 28374)
(1911, 28374)


In [41]:
#SVM using polarity_score
model = LinearSVC(C=1.0, class_weight='balanced', multi_class='ovr', random_state=40)
model.fit(x_train_vec, y_train)
predictions = model.predict(x_test_vec)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.84073507 0.53553719]
recall: [0.79622915 0.60902256]
fscore: [0.81787709 0.56992084]
support: [1379  532]
accuracy: 0.7441130298273155


In [42]:
#LogisticRegression using polarity_score
from sklearn.linear_model import LogisticRegression 
 
classifier = LogisticRegression() 
classifier.fit(x_train_vec, y_train)  

predictions = classifier.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.77189888 0.68571429]
recall: [0.95213923 0.27067669]
fscore: [0.8525974  0.38814016]
support: [1379  532]
accuracy: 0.7624280481423339


/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [43]:
#RandomForest_100 using polarity_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf_100 = RandomForestClassifier(oob_score=True, n_estimators = 100)
rf_100.fit(x_train_vec, y_train)
predictions = rf_100.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74536532 0.84415584]
recall: [0.99129804 0.12218045]
fscore: [0.85091815 0.2134647 ]
support: [1379  532]
accuracy: 0.749345892203035


In [44]:
#KNN using polarity_score
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors=4,weights='distance')
knn_clf.fit(x_train_vec,y_train)
predictions = knn_clf.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74694105 0.68141593]
recall: [0.97389413 0.14473684]
fscore: [0.84545168 0.23875969]
support: [1379  532]
accuracy: 0.7430664573521716


In [45]:
vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, max_features=None, 
                             strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', 
                             ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1)
x_train_vec = vectorizer.fit_transform(x_train).toarray()
print(x_train_vec.shape)
x_test_vec = vectorizer.transform(x_test).toarray()
print(x_test_vec.shape)

ner_list = ['PERSON','LOC','ORG','DATE','TIME','NORP','PRODUCT','EVENT','PERCENT','QUANTITY','ORDINAL','CARDINAL']
features = []
for i, item in enumerate(x_train):
    feature = []
    doc = nlp(item)
    doc = [X.label_ for X in doc.ents]
    for ner in ner_list:
        if ner in doc:
            feature.append(1)
        else:
            feature.append(0)
    features.append(feature)
features = np.array(features)
x_train_vec = np.concatenate((x_train_vec, features), axis=1)


features = []
for i, item in enumerate(x_test):
    feature = []
    doc = nlp(item)
    doc = [X.label_ for X in doc.ents]
    for ner in ner_list:
        if ner in doc:
            feature.append(1)
        else:
            feature.append(0)
    features.append(feature)
features = np.array(features)
x_test_vec = np.concatenate((x_test_vec, features), axis=1)

features = []
for i, item in enumerate(x_train):
    polarity_score = vader_analyzer.polarity_scores(item)
    feature = [score for (name, score) in polarity_score.items()]
    features.append(feature)
    
features = np.array(features)
x_train_vec = np.concatenate((x_train_vec, features), axis=1)
print(x_train_vec.shape)

features = []
for i, item in enumerate(x_test):
    polarity_score = vader_analyzer.polarity_scores(item)
    feature = [score for (name, score) in polarity_score.items()]
    features.append(feature)
    
features = np.array(features)
x_test_vec = np.concatenate((x_test_vec, features), axis=1)
print(x_test_vec.shape)

(11410, 28370)
(1911, 28370)
(11410, 28386)
(1911, 28386)


In [46]:
#SVM using NER & polarity_score
model = LinearSVC(C=1.0, class_weight='balanced', multi_class='ovr', random_state=40)
model.fit(x_train_vec, y_train)
predictions = model.predict(x_test_vec)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.8404908  0.53377265]
recall: [0.79477883 0.60902256]
fscore: [0.8169959  0.56892011]
support: [1379  532]
accuracy: 0.7430664573521716


In [47]:
#LogisticRegression using NER & polarity_score
from sklearn.linear_model import LogisticRegression 
 
classifier = LogisticRegression() 
classifier.fit(x_train_vec, y_train)  

predictions = classifier.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.7742704  0.65948276]
recall: [0.94271211 0.28759398]
fscore: [0.85022891 0.40052356]
support: [1379  532]
accuracy: 0.760334903192046


/Users/jiangqi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [48]:
#RandomForest_100 using NER & polarity_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf_100 = RandomForestClassifier(oob_score=True, n_estimators = 100)
rf_100.fit(x_train_vec, y_train)
predictions = rf_100.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.74726477 0.84337349]
recall: [0.99057288 0.13157895]
fscore: [0.8518865  0.22764228]
support: [1379  532]
accuracy: 0.7514390371533228


In [49]:
#KNN using NER & polarity_score
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors=4,weights='distance')
knn_clf.fit(x_train_vec,y_train)
predictions = knn_clf.predict(x_test_vec) 
precision,recall,fscore,support=precision_recall_fscore_support(y_test, predictions)
print('precision: {0}'.format(precision))
print('recall: {0}'.format(recall))
print('fscore: {0}'.format(fscore))
print('support: {0}'.format(support))
accuracy = accuracy_score(y_test, predictions)
print('accuracy: {0}'.format(accuracy))

precision: [0.76181818 0.53256705]
recall: [0.91153009 0.2612782 ]
fscore: [0.82997689 0.35056747]
support: [1379  532]
accuracy: 0.7305075876504448
